In [ ]:
# Avergae GCP per district
# Number of at_risks students per district
# Engagement rate per district


# Extract District Information
# 001 Washington District
# 002 New York District

# Create a table for districts
#distric_id
# district_name